In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext, Row  
from pyspark.sql import functions as F
sc =SparkContext()
sqlContext=SQLContext(sc)

In [2]:
T= sc.textFile("C:/Users/quartermaine/Documents/Big Data Analytics Labs/temperature-readings-tiny.csv")

### Assignments

#### 1)What are the lowest and highest temperatures measured each year for the period 1950-2014. Provide the lists sorted in the descending order with respect to the maximum temperature. In this exercise you will use the temperature-readings.csv file.¶

In [3]:
lines = T.map(lambda line: line.split(";"))

year_temperature = lines.map(lambda x: (x[1][0:4], float(x[3])))

year_temperature = year_temperature.filter(lambda x: int(x[0]) >= 1950 and int(x[0]) <= 2014)

max_temperatures= year_temperature.reduceByKey(lambda x,y :x if x>=y else y)

max_temperaturesSorted=max_temperatures.sortBy(ascending=False,keyfunc=lambda k:k[1])

max_temps=sqlContext.createDataFrame(max_temperaturesSorted, schema=['Year', 'Temp'])

min_temperatures= year_temperature.reduceByKey(lambda x,y :x if x<=y else y)

min_temperaturesSorted=min_temperatures.sortBy(ascending=False,keyfunc=lambda k:k[1])

min_temps=sqlContext.createDataFrame(min_temperaturesSorted, schema=['Year', 'Temp'])

max_temps.registerTempTable("maxReadingsTable")
min_temps.registerTempTable("minReadingsTable")

In [4]:
print("----Table with max temperatures per year---")
max_temps.show()
print("----Table with min temperatures per year---")
min_temps.show()


----Table with max temperatures per year---
+----+----+
|Year|Temp|
+----+----+
|2014|29.1|
|1960|29.0|
|1959|28.2|
|1958|28.1|
|1956|26.0|
|1957|25.2|
|1955|20.4|
|1961|19.0|
|2013|10.2|
+----+----+

----Table with min temperatures per year---
+----+-----+
|Year| Temp|
+----+-----+
|2013|-13.3|
|1957|-19.9|
|1959|-23.2|
|1961|-23.5|
|2014|-24.3|
|1955|-26.2|
|1958|-27.9|
|1960|-28.3|
|1956|-30.0|
+----+-----+



### Question 1

###  a)

In [5]:
year_temp = lines.map(lambda x: (x[1][0:4],( float(x[3]),x[0])))

year_temp = year_temp.filter(lambda x: int(x[0]) >= 1950 and int(x[0]) <= 2014)

#############################
max_temperatures1= year_temp.reduceByKey(lambda x,y :x if x>=y else y)

max_temperaturesSorted1=max_temperatures1.sortBy(ascending=False,keyfunc=lambda k:k[1][0])

t=max_temperaturesSorted1.map(lambda x: (x[0], float(x[1][0]),x[1][1]))

max_temps1=sqlContext.createDataFrame(t, schema=['Year', 'Temp','station_number'])

#############################

min_temperatures1= year_temp.reduceByKey(lambda x,y :x if x<=y else y)

min_temperaturesSorted1=min_temperatures1.sortBy(ascending=False,keyfunc=lambda k:k[1][0])

t1=min_temperaturesSorted1.map(lambda x: (x[0], float(x[1][0]),x[1][1]))

min_temps1=sqlContext.createDataFrame(t1, schema=['Year', 'Temp','station_number'])


In [6]:
print("----Table with max temperatures per year---")
max_temps1.show()
print("----Table with min temperatures per year---")
min_temps1.show()

----Table with max temperatures per year---
+----+----+--------------+
|Year|Temp|station_number|
+----+----+--------------+
|2014|29.1|        102170|
|1960|29.0|        102190|
|1959|28.2|        102190|
|1958|28.1|        102190|
|1956|26.0|        102190|
|1957|25.2|        102190|
|1955|20.4|        102190|
|1961|19.0|        102190|
|2013|10.2|        102170|
+----+----+--------------+

----Table with min temperatures per year---
+----+-----+--------------+
|Year| Temp|station_number|
+----+-----+--------------+
|2013|-13.3|        102170|
|1957|-19.9|        102190|
|1959|-23.2|        102190|
|1961|-23.5|        102190|
|2014|-24.3|        102170|
|1955|-26.2|        102190|
|1958|-27.9|        102190|
|1960|-28.3|        102190|
|1956|-30.0|        102190|
+----+-----+--------------+



###  b)

In [7]:
import pandas as pd
dat=pd.read_csv("temperature-readings-tiny.csv",sep=';',header=None)
dat.columns = ['station_number', 'date','time','Temperature','quality']
print("first 5 rows")
dat.head(5)
# print("column names")
# dat.columns

first 5 rows


,station_number,date,time,Temperature,quality
0,102170,2013-11-01,06:00:00,6.8,G
1,102170,2013-11-01,18:00:00,3.8,G
2,102170,2013-11-02,06:00:00,5.8,G
3,102170,2013-11-02,18:00:00,-1.1,G
4,102170,2013-11-03,06:00:00,-0.2,G


In [8]:
dt=dat[['station_number','date','Temperature']]
dt['year'] = pd.DatetimeIndex(dt['date']).year
dt.head()
dt.columns

C:\Users\quartermaine\Anaconda2\envs\python3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Index(['station_number', 'date', 'Temperature', 'year'], dtype='object')

In [9]:
DT=dt[['station_number','year','Temperature']]
DT.head()

,station_number,year,Temperature
0,102170,2013,6.8
1,102170,2013,3.8
2,102170,2013,5.8
3,102170,2013,-1.1
4,102170,2013,-0.2


In [10]:
DT.groupby('year').agg({'Temperature': ['min', 'max']})

Temperature      
             min   max
year                  
1955       -26.2  20.4
1956       -30.0  26.0
1957       -19.9  25.2
1958       -27.9  28.1
1959       -23.2  28.2
1960       -28.3  29.0
1961       -23.5  19.0
2013       -13.3  10.2
2014       -24.3  29.1
2015       -19.6  26.8
2016       -25.5  25.8

#### 2)Count the number of readings for each month in the period of 1950-2014 which are higher than 10 degrees.Repeat the exercise,this time taking only distinct readings from each station. That is, if a station reported are a ding above 10 degrees in some month,then it appears only once in the count for that month.In this exercise you will use the temperature-readings.csvfile.¶
#### The out put should contain the following information:Year, month, count


In [11]:
y = lines.map(lambda x: ( (x[1][0:4],x[1][5:7]),float(x[3]) ))

y = y.filter(lambda x: int(x[0][0]) >= 1950 and int(x[0][0]) <= 2014)

#############################
tempsOver= y.filter(lambda x : x[1] >= 10)

tempsOver_unpack=tempsOver.map(lambda x: (x[0][0], x[0][1],float(x[1]) ))

tempsDf=sqlContext.createDataFrame(tempsOver_unpack, schema=['Year', 'Month','Temp'])

In [12]:
tempsDf.groupby("Year", "Month").agg(F.count('Temp').alias("cnt")).orderBy('Year','Month').show()

+----+-----+---+
|Year|Month|cnt|
+----+-----+---+
|1955|   09| 55|
|1955|   10| 23|
|1956|   03|  4|
|1956|   04|  7|
|1956|   05| 60|
|1956|   06| 92|
|1956|   07|108|
|1956|   08| 84|
|1956|   09| 54|
|1956|   10| 17|
|1957|   03|  1|
|1957|   04| 13|
|1957|   05| 46|
|1957|   06| 72|
|1957|   07|109|
|1957|   08| 96|
|1957|   09| 43|
|1957|   10|  9|
|1958|   04|  4|
|1958|   05| 40|
+----+-----+---+
only showing top 20 rows



#### Distinct

In [38]:

tempsOver_unpack1=tempsOver_unpack.distinct()

tempsDf1=sqlContext.createDataFrame(tempsOver_unpack1, schema=['Year', 'Month','Temp'])


In [39]:
tempsDf1.groupby("Year", "Month").agg(F.count('Temp').alias("cnt")).orderBy('Year','Month').show()

+----+-----+---+
|Year|Month|cnt|
+----+-----+---+
|1955|   09| 35|
|1955|   10| 20|
|1956|   03|  4|
|1956|   04|  5|
|1956|   05| 44|
|1956|   06| 58|
|1956|   07| 76|
|1956|   08| 49|
|1956|   09| 30|
|1956|   10| 13|
|1957|   03|  1|
|1957|   04| 13|
|1957|   05| 33|
|1957|   06| 51|
|1957|   07| 69|
|1957|   08| 58|
|1957|   09| 27|
|1957|   10|  8|
|1958|   04|  3|
|1958|   05| 31|
+----+-----+---+
only showing top 20 rows



#### 3)Find the average monthly temperature for each available station in Sweden. Your result should include average temperature for each station for each month in the period of 1960-2014. Bear in mind that not every station has the readings for each month in this timeframe.In this exercise you will use the temperature-readings.csvfile.¶
#### The output should contain the following information: Year, month, station number, average monthly temperature


In [33]:
rdd = lines.map(lambda x: ( (x[1][0:4],x[1][5:7],x[0],x[1][8:10]),float(x[3]) ))

rdd = rdd.filter(lambda x: int(x[0][0]) >= 1960 and int(x[0][0]) <= 2014)

#############################

rdd_filtered=rdd.map(lambda x: (x[0][2],x[0][0], x[0][1],x[0][3],float(x[1]) ))

DF=sqlContext.createDataFrame(rdd_filtered, schema=['Station_number','Year', 'Month','Day','Temp'])

In [34]:
DF.show()

+--------------+----+-----+---+----+
|Station_number|Year|Month|Day|Temp|
+--------------+----+-----+---+----+
|        102170|2013|   11| 01| 6.8|
|        102170|2013|   11| 01| 3.8|
|        102170|2013|   11| 02| 5.8|
|        102170|2013|   11| 02|-1.1|
|        102170|2013|   11| 03|-0.2|
|        102170|2013|   11| 03| 5.6|
|        102170|2013|   11| 04| 6.5|
|        102170|2013|   11| 04| 5.1|
|        102170|2013|   11| 05| 4.2|
|        102170|2013|   11| 05| 3.2|
|        102170|2013|   11| 06| 1.7|
|        102170|2013|   11| 06| 0.9|
|        102170|2013|   11| 07|-0.1|
|        102170|2013|   11| 07| 0.1|
|        102170|2013|   11| 08|-1.2|
|        102170|2013|   11| 08| 5.3|
|        102170|2013|   11| 09| 5.6|
|        102170|2013|   11| 09| 3.8|
|        102170|2013|   11| 10| 2.2|
|        102170|2013|   11| 10|-1.7|
+--------------+----+-----+---+----+
only showing top 20 rows



In [89]:
DT=DF.groupby( "Year","Month",'Day','Station_number')\
.agg( (F.avg('Temp') ).alias("avg_temperature"))\
.orderBy('Station_number','Month','Day')
DT.show()

+----+-----+---+--------------+-------------------+
|Year|Month|Day|Station_number|    avg_temperature|
+----+-----+---+--------------+-------------------+
|2014|   01| 01|        102170|                2.5|
|2014|   01| 02|        102170|               1.75|
|2014|   01| 03|        102170|                2.6|
|2014|   01| 04|        102170|               4.85|
|2014|   01| 05|        102170|               0.85|
|2014|   01| 06|        102170|                1.6|
|2014|   01| 07|        102170|               6.55|
|2014|   01| 08|        102170|  5.550000000000001|
|2014|   01| 09|        102170|                2.4|
|2014|   01| 10|        102170|              -1.65|
|2014|   01| 11|        102170|               -7.8|
|2014|   01| 12|        102170|             -13.95|
|2014|   01| 13|        102170|-15.450000000000001|
|2014|   01| 14|        102170|               -8.0|
|2014|   01| 15|        102170|               -7.0|
|2014|   01| 16|        102170|              -6.45|
|2014|   01|

In [92]:
DT.groupBy('Year','Month','Station_number')\
.agg( (F.sum('avg_temperature')/F.count('avg_temperature') ).alias("avg_temperature"))\
.orderBy('Year','Month','Station_number').show()

+----+-----+--------------+--------------------+
|Year|Month|Station_number|     avg_temperature|
+----+-----+--------------+--------------------+
|1960|   01|        102190|  -6.875000000000001|
|1960|   02|        102190|  -9.618103448275862|
|1960|   03|        102190|  -2.279838709677419|
|1960|   04|        102190|                2.51|
|1960|   05|        102190|  10.768548387096775|
|1960|   06|        102190|  15.769722222222223|
|1960|   07|        102190|  14.205645161290322|
|1960|   08|        102190|   13.41989247311828|
|1960|   09|        102190|   9.402222222222223|
|1960|   10|        102190|  2.0943548387096778|
|1960|   11|        102190| -0.4299999999999999|
|1960|   12|        102190|   -2.42741935483871|
|1961|   01|        102190|  -7.215053763440861|
|1961|   02|        102190| -2.1333333333333337|
|1961|   03|        102190|   2.708602150537634|
|1961|   04|        102190|   4.507777777777778|
|1961|   05|        102190|  10.615333333333332|
|2013|   11|        

#### 4) Provide a list of stations with their associated maximum measured temperatures and maximum measured daily precipitation. Show only those stations where the maximum temperature is between 25 and 30 degrees and maximum daily precipitation is between 100 mm and 200mm.In this exercise you will use the temperature-readings.csvand precipitation-readings.csvfiles.The output should contain the following information: Station number, maximum measured temperature, maximum daily precipitation¶
#### HINT: The correct result for this question should be empty


In [16]:
l = T.map(lambda line: line.split(";"))

station_temperature = l.map(lambda x: (x[0], float(x[3])))

#station_temperature = station_temperature.filter(lambda x: int(x[0]) >= 1950 and int(x[0]) <= 2014)

max_temp_station= station_temperature.reduceByKey(lambda x,y :x if x>=y else y)

max_temp_station=max_temp_station.filter(lambda x: x[1]>=25. and x[1]<=30.)

max_temp_st=max_temp_station.map(lambda x: (x[0],float(x[1]) ))

DataFrame_st=sqlContext.createDataFrame(max_temp_st, schema=['Station_number','Temp'])

In [17]:
DataFrame_st.show()

+--------------+----+
|Station_number|Temp|
+--------------+----+
|        102170|29.1|
|        102190|29.0|
+--------------+----+



In [76]:
prep=sc.textFile("C:/Users/quartermaine/Documents/Big Data Analytics Labs/precipitation-readings.csv")

L=prep.map(lambda line: line.split(";"))

st = L.map(lambda x: (x[0], float(x[3])))

max_st= st.reduceByKey(lambda x,y :x if x>=y else y)

max_st=max_st.filter(lambda x: x[1]>=10. and x[1]<=20.)

wst=max_st.map(lambda x: (x[0],float(x[1]) ))

DataFrame_prep=sqlContext.createDataFrame(wst, schema=['Station_number','precipitation'])

DataFrame_prep.show()

+--------------+-------------+
|Station_number|precipitation|
+--------------+-------------+
|         77210|         18.9|
|        144310|         12.8|
|        162860|         12.0|
|        170860|         20.0|
|        107140|         19.2|
|        154860|         18.0|
|        182810|         19.6|
|        169790|         11.0|
|        123060|         17.0|
|        167990|         15.7|
|        188790|         19.1|
|        103420|         16.5|
|        178860|         12.2|
|         78400|         14.0|
|         85240|         18.0|
|        114410|         19.0|
|        189720|         19.2|
|         99280|         18.4|
|        132170|         12.5|
|        145130|         18.0|
+--------------+-------------+
only showing top 20 rows



In [77]:
inner_join = DataFrame_st.join(DataFrame_prep, DataFrame_st.Station_number == DataFrame_prep.Station_number)
print("----Table with max temperatures and precipitation---")
inner_join.show()

----Table with max temperatures and precipitation---
+--------------+----+--------------+-------------+
|Station_number|Temp|Station_number|precipitation|
+--------------+----+--------------+-------------+
+--------------+----+--------------+-------------+



#### 5)Calculate the average monthly precipitation for the Östergotland region (list of stations is provided in the separate file)for the period1993-2016. In order to do this, you will first need to calculate the total monthly precipitation for each station before calculating the monthly average (by averaging over stations).In this exercise you will use the precipitation-readings.csvand stations-Ostergotland.csv files. HINT(not for the SparkSQL lab): Avoid using joins here! stations-Ostergotland.csv is small and if distributed will cause a number of unnecessary shuffles when joined with precipitationRDD. If you distribute precipitation-readings.csv then either repartition your stations RDD to 1 partition or make use of the collect to acquire a python list and broadcast function to broadcast the list to all nodes.¶
#### The output should contain the following information: Year,month,average monthly precipitation


In [65]:
stations=sc.textFile("C:/Users/quartermaine/Documents/Big Data Analytics Labs/stations-Ostergotland.csv")
precipitation=sc.textFile("C:/Users/quartermaine/Documents/Big Data Analytics Labs/precipitation-readings.csv")

stations=stations.map(lambda line: line.split(";"))
precipitation=precipitation.map(lambda line:line.split(";"))

df_precipitation=sqlContext.createDataFrame(precipitation, schema=['Station_number',
                                                     'Date','Time','precipitation','quality'])

df_stations=sqlContext.createDataFrame(stations, schema=['Station_number','Station_name','Measurment_height',
                                                         'Latitude','Longtitude','Readings_from','Readings_to',
                                                        'Elevation'])


In [66]:
df_precipitation.show()

+--------------+----------+--------+-------------+-------+
|Station_number|      Date|    Time|precipitation|quality|
+--------------+----------+--------+-------------+-------+
|        103100|1995-08-01|00:00:00|          0.0|      Y|
|        103100|1995-08-01|01:00:00|          0.0|      Y|
|        103100|1995-08-01|02:00:00|          0.0|      Y|
|        103100|1995-08-01|03:00:00|          0.0|      Y|
|        103100|1995-08-01|04:00:00|          0.0|      Y|
|        103100|1995-08-01|05:00:00|          0.0|      Y|
|        103100|1995-08-01|06:00:00|          0.0|      Y|
|        103100|1995-08-01|07:00:00|          0.0|      Y|
|        103100|1995-08-01|08:00:00|          0.0|      Y|
|        103100|1995-08-01|09:00:00|          0.0|      Y|
|        103100|1995-08-01|10:00:00|          0.0|      Y|
|        103100|1995-08-01|11:00:00|          0.0|      Y|
|        103100|1995-08-01|12:00:00|          0.0|      Y|
|        103100|1995-08-01|13:00:00|          0.0|      

In [67]:
df_stations.show()

+--------------+--------------------+-----------------+--------+----------+-------------------+-------------------+---------+
|Station_number|        Station_name|Measurment_height|Latitude|Longtitude|      Readings_from|        Readings_to|Elevation|
+--------------+--------------------+-----------------+--------+----------+-------------------+-------------------+---------+
|         84260|             Örberga|              2.0| 58.4274|    14.826|1990-11-01 00:00:00|1993-03-31 23:59:59|    105.0|
|         85630|           Västra Ny|              2.0|   58.65|     15.05|1961-01-01 00:00:00|1966-12-31 23:59:59|    120.0|
|         85460|         Kettstaka A|              2.0| 58.7165|     15.03|1995-08-01 00:00:00|2016-10-01 08:00:00|    225.0|
|         85450|            Godegård|              2.0| 58.7936|   15.1694|1961-01-01 00:00:00|1988-02-29 23:59:59|    130.0|
|         85490|          Zinkgruvan|              2.0| 58.8063|   15.1285|1983-09-01 00:00:00|2016-08-31 23:59:59|   

In [68]:
D_grouped=df_precipitation.groupBy(F.year(df_precipitation.Date),F.month(df_precipitation.Date),df_precipitation.Station_number)\
.agg(F.avg(df_precipitation.precipitation).alias("avg_precipitation"))
D_grouped.show()

+----------+-----------+--------------+--------------------+
|year(Date)|month(Date)|Station_number|   avg_precipitation|
+----------+-----------+--------------+--------------------+
|      2008|         11|        103100|  0.0844444444444445|
|      2013|          5|        103100| 0.03454301075268818|
|      2015|          1|        103100|  0.1138440860215054|
|      2015|          6|        103100| 0.11944444444444446|
|      2011|          8|        104580| 0.12002688172043008|
|      2014|          1|        104580| 0.07217741935483882|
|      2015|          9|        105220|  0.2039829302987195|
|      2016|          1|        105220| 0.08301886792452835|
|      2010|          3|        106570|0.032436069986541055|
|      2012|          3|        106570|0.007258064516129033|
|      2004|          1|        107420|0.057997218358831724|
|      2007|          6|        107420| 0.07138888888888892|
|      2005|          4|        114140| 0.02743732590529248|
|      2011|          3|

In [75]:
from pyspark.sql.functions import broadcast
df_stations1=df_stations.select('Station_number')
D_grouped.join(broadcast(df_stations1), D_grouped.Station_number == df_stations1.Station_number)\
.orderBy('year(Date)').show()

+----------+-----------+--------------+--------------------+--------------+
|year(Date)|month(Date)|Station_number|   avg_precipitation|Station_number|
+----------+-----------+--------------+--------------------+--------------+
|      1993|          9|         86340| 0.06023738872403561|         86340|
|      1993|         10|         86340|0.058142664872139975|         86340|
|      1993|          4|         86340|                 0.0|         86340|
|      1993|          7|         86340| 0.12874493927125505|         86340|
|      1993|          6|         86340| 0.09323432343234324|         86340|
|      1993|         12|         86340|  0.0516713091922006|         86340|
|      1993|          5|         86340| 0.02851351351351352|         86340|
|      1993|         11|         86340|0.060537482319660586|         86340|
|      1993|          8|         86340| 0.11414427157001421|         86340|
|      1994|          2|         86340| 0.03393665158371041|         86340|
|      1994|

#### 6)Compare the average monthly temperature(find the difference)in the period1950-2014 for all stations in Östergotland with long-term monthly averages in the period of 1950-1980. Make a plot of your results.
#### HINT:The first step is to find the monthly averages for each station.In the next step,you can average over all stations to acquire the average temperature for a specific year and month. This  RDD/DataFrame can be used to compute the long-term average by averaging over all the years in the interval.The output should contain the following information: Year, month,difference